In [1]:
#Import required packages 
import numpy as np

import pandas as pd
import pandas_profiling

from datetime import datetime

import pickle

# Import Bitcoin Scrapped Dataset and Preprocess

In [2]:
#Import Bitcoin Barchart Data
bitcoin_price_history_df = pd.read_csv('./data/price_history_4_year_hourly.csv')

### Note: All financial columns are in U.S Dollars
- Open  
- Close 
- Low   
- High  
- Volume (Currency) 
- Weighted Price

In [7]:
bitcoin_price_history_df.sample(5)

,timestamp,open,high,low,close,volume (btc),volume (currency),weighted price
14668,2017-02-03 00:00:00,1009.15,1009.93,1004.37,1008.00,130.40,131309.57,1007.00
8267,2016-05-10 09:00:00,459.00,459.60,453.10,454.88,457.00,208292.27,455.78
8151,2016-05-05 13:00:00,446.86,448.44,446.02,448.44,153.93,68827.37,447.14
33806,2019-04-11 06:00:00,5219.90,5224.99,5200.00,5220.00,405.92,2116117.80,5213.12
34911,2019-05-27 07:00:00,8757.73,8815.86,8750.00,8778.72,445.71,3915796.22,8785.47


In [4]:
#strip column names of any space and lowercase
bitcoin_price_history_df.rename(columns=lambda x: x.strip())
bitcoin_price_history_df.columns = bitcoin_price_history_df.columns.str.lower()

#Reconfigure Date to DateTime
bitcoin_price_history_df.timestamp = pd.to_datetime(bitcoin_price_history_df.timestamp, format="%m/%d/%Y %H:%M")

In [5]:
#Verify All Column DataTypes Are Int64
bitcoin_price_history_df.dtypes

timestamp            datetime64[ns]
open                        float64
high                        float64
low                         float64
close                       float64
volume (btc)                float64
volume (currency)           float64
weighted price              float64
dtype: object

In [6]:
# #Save Updated Data Frame
with open('./data/bitcoin_4_year_hourly_history_df.pickle', 'wb') as file:
     pickle.dump(bitcoin_price_history_df, file)     